# COGS 108 - EDA Checkpoint

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Setup

In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None #suppressing SettingWithCopyWarning: used to not set a value

# Data Cleaning

In [3]:
years = ['2012','2013','2014','2015','2016','2017',
'2018','2019','2020','2021','2022'] # years to be observed

#reading csv
weo = pd.read_csv('WEO.csv')
happiness = pd.read_csv('Happiness.csv')

#using set to extract common names
weo_country = set(weo["Country"].unique().tolist())
happiness_country = set(happiness["Country name"].unique().tolist())

common_country = weo_country & happiness_country
weo_not = weo_country - common_country
happiness_not = happiness_country - common_country

print(sorted(happiness_not))
print(sorted(weo_not))

['Congo (Brazzaville)', 'Congo (Kinshasa)', 'Cuba', 'Czechia', 'Gambia', 'Hong Kong S.A.R. of China', 'Iran', 'Ivory Coast', 'Kyrgyzstan', 'Laos', 'Slovakia', 'Somaliland region', 'South Korea', 'State of Palestine', 'Syria', 'Turkiye']
['Andorra', 'Antigua and Barbuda', 'Aruba', 'Barbados', 'Brunei Darussalam', 'Cabo Verde', 'Czech Republic', "CÙte d'Ivoire", 'Democratic Republic of the Congo', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'Fiji', 'Grenada', 'Guinea-Bissau', 'Hong Kong SAR', 'Islamic Republic of Iran', 'Kiribati', 'Korea', 'Kyrgyz Republic', 'Lao P.D.R.', 'Macao SAR', 'Marshall Islands', 'Micronesia', 'Nauru', 'Palau', 'Papua New Guinea', 'Puerto Rico', 'Republic of Congo', 'Samoa', 'San Marino', 'Seychelles', 'Slovak Republic', 'Solomon Islands', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines', 'S„o TomÈ and PrÌncipe', 'The Bahamas', 'The Gambia', 'Timor-Leste', 'Tonga', 'Tuvalu', 'T¸rkiye', 'Vanuatu', 'West Bank and Gaza']


Above is the lists of countries that are not common from both sets of data. 
First is from the Happiness Report and second is from the WEO.

Since some country names are reported different in each of the organizations, we manually matched such countries: 

* Congo (Brazzaville) -> Republic of Congo
* Congo (Kinshasa) -> Democratic Republic of the Congo
* Czechia -> Czech Republic
* Hong Kong S.A.R of China -> Hong Kong SAR
* Iran -> Iran (from Islamic Republic of Iran)
* Kyrgyzstan -> Kyrgyzstan (from Kyrgyz Republic)
* Laos -> Laos (from Lao P.D.R)
* Slovakia -> Slovakia (from Slovak Republic)
* South Korea -> South Korea (from Korea)
* Turkiye -> Turkiye (from T¸rkiye)
* Cuba, Gambia, Ivory Coast, Somaliland region, State of Palestine, Syria - > removed

In [4]:
#replacing left to right
happiness['Country name'].replace(['Congo (Brazzaville)', 'Congo (Kinshasa)', 'Czechia', 'Hong Kong S.A.R. of China'], 
            ['Republic of Congo', 'Democratic Republic of the Congo', 'Czech Republic', 'Hong Kong SAR'], inplace = True)
#replacing right to (from [name])
weo['Country'].replace(['Islamic Republic of Iran', 'Kyrgyz Republic', 'Lao P.D.R.', 'Slovak Republic', 'Korea', 'T¸rkiye'],
                 ['Iran', 'Kyrgyzstan', 'Laos', 'Slovakia', 'South Korea', 'Turkiye'], inplace = True)

#checking
weo_country = set(weo["Country"].unique().tolist())
happiness_country = set(happiness["Country name"].unique().tolist())
common_country = list(sorted(weo_country & happiness_country))
print(common_country)

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong SAR', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania', 'Luxembou

The above is the list of countries that are common from both of the lists. 

Note that these countries may not have all entries for variables we are observing and are subjected to be excluded in the later process.

From here on, a single data frame: `data` containing entries from both dataframe will be constructed.

In [5]:
#creating unified data frame: data

#dropping other variables
data = happiness[['Country name', 'year', 'Life Ladder', 'Positive affect', 'Negative affect']] 
#only keeping those after 2012
data.drop(data[data['year'] < 2012].index, inplace = True) 
#numerical year value is no longer needed / potential to influence other num vals
data['year'] = data['year'].apply(str) 

#adding weo variables to data
weo_vars = weo['Subject Descriptor'].unique().tolist()
for var in weo_vars:
    data[var] = np.nan
    
#copying of weo values to data
weo = weo.drop(columns = ['Subject Notes','Units', 'Scale' , 'Country/Series-specific Notes', 'Estimates Start After'])
index = data.index #optain all indices of data
for i in index:
    country = data.loc[i,'Country name'] #get country name from index i
    year = data.loc[i,'year'] #year from index i
    for var in weo_vars: 
        if country in common_country and year in years:
            #accessing values of variables from weo
            temp = weo.loc[(weo['Country'] == country)&(weo['Subject Descriptor'] == var) , year] 
            if temp.empty == False:
                #writing the values of variables from weo to data
                data.at[i,var] = temp[temp.index[0]]

The `data` variable now contains all information from two tables from the years 2012 - 2022 with common countries. 

Some entries of WEO report is missing and are indicated with `string` object `'na'`. 

The following is the cleaning of all invalid inputs.

In [6]:
#'na' to 'NaN' along with changing values to float of WEO variable columns
cols = data.columns.drop(['Country name', 'year'])
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')

#dropping nans
data = data.dropna()

#index reset
data.reset_index(drop=True, inplace=True)

#column renaming
data.columns = [ 'Country', 'Year', 'LifeLadder', 'PositiveAffect' , 'NegativeAffect',
              'GDP', 'Investment','Savings','CPI','VIGS','VIG','VEGS', 
              'VEG', 'Population', 'Revenue','Expenditure','Debt','Balance','Unemployment']

In [7]:
data.sample(5)

,Country,Year,LifeLadder,PositiveAffect,NegativeAffect,GDP,Investment,Savings,CPI,VIGS,VIG,VEGS,VEG,Population,Revenue,Expenditure,Debt,Balance,Unemployment
505,Nicaragua,2022,6.392,0.787,0.339,15.770,23.513,21.295,271.905,8.118,7.356,2.320,-3.158,6.607,27.039,29.015,45.956,-2.219,7.518
616,Serbia,2012,5.155,0.447,0.371,43.300,19.269,8.474,163.114,0.990,0.753,-0.073,-0.836,7.201,38.771,45.131,53.869,-10.795,24.600
252,Finland,2018,7.858,0.749,0.182,275.833,25.237,23.392,102.415,5.732,3.843,1.503,0.815,5.513,52.526,53.380,64.863,-1.845,7.425
304,Hungary,2020,6.038,0.662,0.240,157.182,27.141,26.121,125.812,-3.855,-2.009,-6.148,-1.110,9.770,43.547,51.083,79.297,-1.133,4.159
653,Slovenia,2021,6.761,0.643,0.261,61.790,21.894,25.697,106.766,17.585,17.245,14.500,13.378,2.109,44.620,49.293,74.466,3.803,4.725


In [8]:
data.describe()

,LifeLadder,PositiveAffect,NegativeAffect,GDP,Investment,Savings,CPI,VIGS,VIG,VEGS,VEG,Population,Revenue,Expenditure,Debt,Balance,Unemployment
count,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000
mean,5.968363,0.671498,0.263817,219.966346,24.441437,23.566118,135.305315,4.027429,3.690104,4.118195,3.747952,23.460494,31.389645,34.015018,53.996931,-0.948911,8.572732
std,0.893949,0.108761,0.081499,226.352593,6.680889,7.947953,93.045971,9.265745,9.087247,10.495230,9.287153,39.409880,11.722681,10.956509,31.033564,7.048594,5.804103
min,3.559000,0.297000,0.083000,1.784000,11.892000,-3.957000,40.194000,-29.230000,-25.916000,-39.078000,-29.748000,0.320000,5.117000,9.761000,0.055000,-43.825000,0.700000
25%,5.311500,0.583000,0.203000,37.499000,20.110000,18.159000,99.128000,-0.184500,-0.807000,0.092500,-0.802000,4.298500,22.350000,25.518000,35.497000,-4.571500,4.958000
50%,5.953000,0.698000,0.256000,128.814000,23.618000,23.225000,104.422000,3.937000,3.602000,3.992000,3.304000,9.051000,29.408000,33.523000,49.273000,-1.397000,6.842000
75%,6.551000,0.759000,0.314000,345.625500,27.251500,27.836000,127.940500,8.530000,8.293000,8.337500,7.708500,23.325000,40.247500,41.894500,68.989000,2.584500,10.150000
max,7.889000,0.884000,0.552000,993.681000,56.417000,58.697000,918.961000,48.744000,42.502000,84.109000,64.411000,258.497000,71.975000,60.267000,212.388000,45.462000,35.300000


# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [24]:
# some idea for extracting which countries do not have data for certain year
# can develop function to spit out a subset of dataframe 
# that only include countries that have data for all year between two given values of years
country_set = set(data["Country"].unique().tolist())
set1 = set(data[(data["Year"]) == "2012"].Country.unique().tolist())
set2 = country_set - set1
print(set2)

{'Bhutan', 'Singapore', 'Uruguay', 'Belize', 'Mauritius', 'Jamaica'}
